In [13]:
import pandas as pd
import unicodedata

df = pd.read_csv("./data_labeled.csv")   # đường dẫn tùy thư mục của bạn

# ==== Hàm bỏ dấu ====
def strip_accents(text: str) -> str:
    text = unicodedata.normalize('NFD', text)
    return ''.join(ch for ch in text if unicodedata.category(ch) != 'Mn')

# ==== Chuẩn hóa LOCATION ====
def normalize_location(value: str) -> str:
    v = strip_accents(str(value)).lower()

    if any(x in v for x in ["hcm", "ho chi minh", "tphcm", "tp.hcm", "tp hochiminh"]):
        return "Thành phố Hồ Chí Minh"
    if "binh duong" in v:
        return "Tỉnh Bình Dương"
    if "dong nai" in v or "nhon trach" in v or "long thanh" in v:
        return "Tỉnh Đồng Nai"
    if "ba ria" in v or "vung tau" in v:
        return "Tỉnh Bà Rịa - Vũng Tàu"
    return "Khác"

# ==== Chuẩn hóa BUSINESS TYPE ====
def normalize_business(value: str) -> str:
    v = strip_accents(str(value)).lower()
    if "hang tau" in v or "hãng tàu" in v:
        return "Hãng tàu"
    if "dai ly" in v or "đại lý" in v:
        return "Đại lý vận tải"
    if "xuat nhap khau" in v:
        return "Công ty xuất nhập khẩu"
    if "san xuat" in v or "cung cap dich vu" in v:
        return "Công ty chuyên sản xuất/ cung cấp dịch vụ"
    return "Khác"

# ==== Chuẩn hóa USAGE FREQUENCY ====
def normalize_usage(value: str) -> str:
    v = strip_accents(str(value)).lower().strip()
    if "1 - 3" in v:
        return "1 - 3 lần/tháng"
    if "tren 3" in v:
        return "Trên 3 lần/tháng"
    return value  # fallback

df["location_group"] = df["location"].apply(normalize_location)
df["business_type_group"] = df["business_type"].apply(normalize_business)
df["usage_group"] = df["usage_frequency"].apply(normalize_usage)


In [14]:
def frequency_table(series: pd.Series) -> pd.DataFrame:
    freq = series.value_counts().reset_index()
    freq.columns = ["category", "frequency"]
    freq["percent"] = (freq["frequency"] / freq["frequency"].sum() * 100).round(1)
    return freq


In [15]:
freq_location = frequency_table(df["location_group"])
freq_business = frequency_table(df["business_type_group"])
freq_usage = frequency_table(df["usage_group"])

print("📌 Địa điểm đăng ký kinh doanh:\n", freq_location, "\n")
print("📌 Loại hình kinh doanh:\n", freq_business, "\n")
print("📌 Tần suất sử dụng dịch vụ:\n", freq_usage, "\n")


📌 Địa điểm đăng ký kinh doanh:
                  category  frequency  percent
0   Thành phố Hồ Chí Minh         90     55.6
1                    Khác         58     35.8
2           Tỉnh Đồng Nai         11      6.8
3  Tỉnh Bà Rịa - Vũng Tàu          2      1.2
4         Tỉnh Bình Dương          1      0.6 

📌 Loại hình kinh doanh:
                                     category  frequency  percent
0                     Công ty xuất nhập khẩu        112     69.1
1                                       Khác         43     26.5
2  Công ty chuyên sản xuất/ cung cấp dịch vụ          5      3.1
3                                   Hãng tàu          2      1.2 

📌 Tần suất sử dụng dịch vụ:
            category  frequency  percent
0  Trên 3 lần/tháng        134     82.7
1   1 - 3 lần/tháng         28     17.3 



In [16]:
with pd.ExcelWriter("task1_frequency_tables.xlsx") as writer:
    freq_location.to_excel(writer, sheet_name="location", index=False)
    freq_business.to_excel(writer, sheet_name="business_type", index=False)
    freq_usage.to_excel(writer, sheet_name="usage", index=False)

print("✅ Đã xuất file: task1_frequency_tables.xlsx")


✅ Đã xuất file: task1_frequency_tables.xlsx


In [11]:
len(df), df["location_group"].isna().sum(), df["business_type_group"].isna().sum(), df["usage_group"].isna().sum()


(162, 0, 0, 0)

In [12]:
df["usage_frequency"].value_counts()


usage_frequency
Trên 3 lần/tháng    134
1 - 3 lần/tháng      28
Name: count, dtype: int64

In [ ]:
def frequency_table(series: pd.Series) -> pd.DataFrame:
    """
    Trả về bảng tần số & tỉ lệ %
    """
    counts = series.value_counts().reset_index()
    counts.columns = ["category", "frequency"]
    total = counts["frequency"].sum()
    counts["percent"] = counts["frequency"] / total * 100
    # sắp xếp theo frequency giảm dần (hoặc giữ nguyên theo index nếu muốn)
    return counts


In [25]:
import pandas as pd
import unicodedata
import numpy as np
from pathlib import Path
import os

df = pd.read_csv("../data_labeled.csv")   # đường dẫn tùy thư mục của bạn



print("Số dòng, số cột:", df.shape)
df.head()
BASE_DIR = Path().resolve()


Số dòng, số cột: (162, 43)


In [26]:
# 9 cột đầu là thông tin mô tả, từ RE1 trở đi là Likert
METADATA_COLS = [
    "timestamp", "email", "company", "location",
    "business_type", "position", "experience",
    "used_service", "usage_frequency"
]

# TẤT CẢ các item thang đo (đúng theo header hiện tại)
ITEM_COLS = [
    "RE1", "RE2", "RE3", "RE4", "RE5",          # Nguồn lực
    "OU1", "OU2", "OU3", "OU4", "OU5",          # Kết quả
    "PR1", "PR2", "PR3", "PR4",                 # Quy trình
    "MA1", "MA2", "MA3", "MA4", "MA5", "MA6",   # Quản lý
    "ISR1", "ISR2", "ISR3",                     # Hình ảnh (phần 1)
    "P1", "P2", "P3", "P4", "P5", "P6",         # Hình ảnh / TNXH (phần 2)
    "CS1", "CS2", "CS3", "CS4", "CS5"           # HL + CS (CS4, CS5 = biến phụ thuộc)
]

# Kiểm tra xem tất cả item đều có trong df chưa
missing_cols = [c for c in ITEM_COLS if c not in df.columns]
print("Thiếu cột nào không? ->", missing_cols)

# Ép kiểu số cho toàn bộ item (phòng trường hợp đọc vào là object/string)
df[ITEM_COLS] = df[ITEM_COLS].apply(pd.to_numeric, errors="coerce")

df[ITEM_COLS].dtypes.head()


Thiếu cột nào không? -> []


RE1    int64
RE2    int64
RE3    int64
RE4    int64
RE5    int64
dtype: object

In [ ]:
# SCALES = {
#     "NL": ["RE1", "RE2", "RE3", "RE4", "RE5"],          # Nguồn lực
#     "KQ": ["OU1", "OU2", "OU3", "OU4", "OU5"],          # Kết quả
#     "QT": ["PR1", "PR2", "PR3", "PR4"],                 # Quy trình
#     "QL": ["MA1", "MA2", "MA3", "MA4", "MA5", "MA6"],   # Quản lý
#     "HA": ["ISR1", "ISR2", "ISR3", "P1", "P2", "P3", "P4", "P5", "P6"],  # Hình ảnh & TNXH
#     "CS": ["CS1", "CS2", "CS3", "CS4", "CS5"]           # Hài lòng khách hàng (biến phụ thuộc)
# }


SCALES = {
    # 6 biến độc lập
    "RE"  : ["RE1", "RE2", "RE3", "RE4", "RE5"],              # Resource
    "OU"  : ["OU1", "OU2", "OU3", "OU4", "OU5"],              # Outcome
    "PR"  : ["PR1", "PR2", "PR3", "PR4"],                     # Process
    "MA"  : ["MA1", "MA2", "MA3", "MA4", "MA5", "MA6"],      # Management
    "ISR" : ["ISR1", "ISR2", "ISR3"],                         # Image & Social responsibility
    "P"   : ["P1", "P2", "P3", "P4", "P5", "P6"],             # Price

    # 1 biến phụ thuộc
    "CS"  : ["CS1", "CS2", "CS3", "CS4", "CS5"]               # Customer satisfaction
}

NL: 5 items -> ['RE1', 'RE2', 'RE3', 'RE4', 'RE5']
KQ: 5 items -> ['OU1', 'OU2', 'OU3', 'OU4', 'OU5']
QT: 4 items -> ['PR1', 'PR2', 'PR3', 'PR4']
QL: 6 items -> ['MA1', 'MA2', 'MA3', 'MA4', 'MA5', 'MA6']
HA: 9 items -> ['ISR1', 'ISR2', 'ISR3', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6']
HL: 3 items -> ['CS1', 'CS2', 'CS3']
CS: 2 items -> ['CS4', 'CS5']


In [28]:
# Tính thống kê mô tả cho từng item
desc_items = df[ITEM_COLS].agg(['count', 'min', 'max', 'mean', 'std']).T

# Đổi tên cột cho giống luận văn
desc_items = desc_items.rename(columns={
    'count': 'N',
    'min': 'Min',
    'max': 'Max',
    'mean': 'Mean',
    'std': 'Std'
})

# Làm tròn 3 chữ số thập phân (giống SPSS)
desc_items = desc_items.round(3)

desc_items.head(10)


,N,Min,Max,Mean,Std
RE1,162.0,1.0,5.0,4.117,0.644
RE2,162.0,2.0,5.0,4.025,0.747
RE3,162.0,2.0,5.0,4.080,0.739
RE4,162.0,1.0,5.0,4.241,0.703
RE5,162.0,1.0,5.0,4.080,0.804
OU1,162.0,1.0,5.0,4.000,0.772
OU2,162.0,2.0,5.0,4.111,0.696
OU3,162.0,1.0,5.0,4.173,0.727
OU4,162.0,2.0,5.0,4.173,0.674
OU5,162.0,1.0,5.0,4.142,0.730


In [29]:
# Tạo cột điểm trung bình cho từng thang đo
for latent, items in SCALES.items():
    df[latent] = df[items].mean(axis=1)

# Thống kê mô tả cho từng thang đo
scale_cols = list(SCALES.keys())
desc_scales = df[scale_cols].agg(['count', 'min', 'max', 'mean', 'std']).T
desc_scales = desc_scales.rename(columns={
    'count': 'N',
    'min': 'Min',
    'max': 'Max',
    'mean': 'Mean',
    'std': 'Std'
}).round(3)

desc_scales


,N,Min,Max,Mean,Std
NL,162.0,1.400,5.0,4.109,0.569
KQ,162.0,2.000,5.0,4.120,0.565
QT,162.0,1.750,5.0,4.131,0.579
QL,162.0,1.833,5.0,4.156,0.541
HA,162.0,2.000,5.0,4.111,0.536
HL,162.0,2.000,5.0,4.148,0.565
CS,162.0,2.000,5.0,4.179,0.567


In [31]:
OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(exist_ok=True)

output_file = OUTPUT_DIR / "Task2_Descriptive_Statistics.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    desc_items.to_excel(writer, sheet_name="Items_Descriptive")
    desc_scales.to_excel(writer, sheet_name="Scales_Descriptive")

print("Đã lưu file Task 2 tại:", output_file)


Đã lưu file Task 2 tại: C:\Users\tranh\Downloads\FTU_Thesis\tasks\output\Task2_Descriptive_Statistics.xlsx


# Task 3

In [44]:
# Ô 1: Cài (nếu chưa có) và import thư viện + đọc dữ liệu

# Chỉ chạy pip một lần trong môi trường mới
# !pip install plspm

import pandas as pd
import plspm.config as c
from plspm.plspm import Plspm
from plspm.scheme import Scheme
from plspm.mode import Mode
from plspm.scale import Scale

# Đọc dữ liệu đã gắn nhãn
# Nếu bạn lưu dạng csv thì dùng read_csv tương ứng
df_raw = pd.read_excel("../data_labeled.xlsx")

# Giữ lại đúng mẫu nghiên cứu: những người "Đã từng sử dụng" dịch vụ
df = df_raw[df_raw["used_service"] == "Đã từng sử dụng"].reset_index(drop=True)

print("Số mẫu sau lọc:", len(df))
df.head()


Số mẫu sau lọc: 159


,timestamp,email,company,location,business_type,position,experience,used_service,usage_frequency,RE1,...,P2,P3,P4,P5,P6,CS1,CS2,CS3,CS4,CS5
0,11/6/2025 9:18:52,ducthach120990@gmail.com,Ecu worldwide,Kho 5,Công ty giao nhận vận tải (Forwader),Nhân viên hiện trường,Trên 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,4,...,3,3,3,3,3,4,4,4,4,4
1,11/6/2025 9:20:20,hiepanctancang@gmail.com,TNHH DỊCH VỤ và VẬN TẢI THẾ GIỚI CHÍNH PHƯƠNG,"Tòa nhà Sky Center, CH2A, Tầng trệt, Block C, ...",Công ty xuất nhập khẩu,Nhân viên hiện trường,Trên 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,5,...,5,5,5,5,4,5,5,5,5,5
2,11/6/2025 9:25:51,huunguyen1801@gmail.com,Shipco Transport Việt Nam,01 đinh lễ quận 4,Công ty giao nhận vận tải (Forwader),Nhân viên hiện trường,Trên 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,5,...,5,5,5,5,5,5,5,5,5,5
3,11/6/2025 13:42:17,nguyenketoan1@gmail.com,Melody,Số 1 nguyễn văn đậu,Công ty giao nhận vận tải (Forwader),Nhân viên hiện trường,1 - 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,5,...,5,5,4,5,4,5,5,5,5,5
4,11/6/2025 13:42:50,duongtran.ab91@gmail.com,Ecu worldwide,Tp hochiminh,Công ty giao nhận vận tải (Forwader),Nhân viên hiện trường,Trên 3 năm,Đã từng sử dụng,Trên 3 lần/tháng,5,...,4,4,4,4,4,4,4,4,4,4


In [45]:
# Ô 2: Khai báo mapping thang đo dùng cho PLS-SEM (Task 3)

SCALES_TASK3 = {
    "NL": ["RE1", "RE2", "RE3", "RE4", "RE5"],                  # Nguồn lực
    "KQ": ["OU1", "OU2", "OU3", "OU4", "OU5"],                  # Kết quả
    "QT": ["PR1", "PR2", "PR3", "PR4"],                         # Quy trình
    "QL": ["MA1", "MA2", "MA3", "MA4", "MA5", "MA6"],          # Quản lý
    "HA": ["ISR1", "ISR2", "ISR3", "P1", "P2", "P3", "P4", "P5", "P6"],  # Hình ảnh & TNXH
    "HL": ["CS1", "CS2", "CS3"],                               # Hài lòng
    "CS": ["CS4", "CS5"],                                      # Lòng trung thành (biến phụ thuộc)
}

# Flatten ra list tất cả các biến quan sát dùng trong mô hình
all_items = [item for items in SCALES_TASK3.values() for item in items]

# DataFrame chỉ chứa các cột item (tất cả đều là số)
df_items = df[all_items].copy()

print("Số biến quan sát:", len(all_items))
print("Kích thước df_items:", df_items.shape)
print("Số missing mỗi cột:\n", df_items.isna().sum())


Số biến quan sát: 34
Kích thước df_items: (159, 34)
Số missing mỗi cột:
 RE1     0
RE2     0
RE3     0
RE4     0
RE5     0
OU1     0
OU2     0
OU3     0
OU4     0
OU5     0
PR1     0
PR2     0
PR3     0
PR4     0
MA1     0
MA2     0
MA3     0
MA4     0
MA5     0
MA6     0
ISR1    0
ISR2    0
ISR3    0
P1      0
P2      0
P3      0
P4      0
P5      0
P6      0
CS1     0
CS2     0
CS3     0
CS4     0
CS5     0
dtype: int64


In [46]:
# Ô 3: Định nghĩa mô hình cấu trúc (đường dẫn giữa các biến tiềm ẩn)

structure = c.Structure()

# Các yếu tố NL, KQ, QT, QL, HA -> ảnh hưởng HL (Hài lòng)
structure.add_path(["NL", "KQ", "QT", "QL", "HA"], ["HL"])

# Các yếu tố NL, KQ, QT, QL, HA, HL -> ảnh hưởng CS (Lòng trung thành)
structure.add_path(["NL", "KQ", "QT", "QL", "HA", "HL"], ["CS"])

# Xem ma trận đường dẫn
path_matrix = structure.path()
path_matrix


,HA,QL,QT,KQ,NL,HL,CS
HA,0,0,0,0,0,0,0
QL,0,0,0,0,0,0,0
QT,0,0,0,0,0,0,0
KQ,0,0,0,0,0,0,0
NL,0,0,0,0,0,0,0
HL,1,1,1,1,1,0,0
CS,1,1,1,1,1,1,0


In [48]:
# Ô 4: Tạo Config cho plspm (outer model)

cfg = c.Config(path_matrix, default_scale=Scale.NUM)

# Thêm từng biến tiềm ẩn (latent variable) với Mode A (phản xạ)
for lv_name, indicators in SCALES_TASK3.items():
    cfg.add_lv(
        lv_name,
        Mode.A,
        *[c.MV(col) for col in indicators]
    )

print("Đã khai báo xong Config cho", len(SCALES_TASK3), "biến tiềm ẩn.")


Đã khai báo xong Config cho 7 biến tiềm ẩn.


In [51]:
# Ô 5: Chạy PLS-PM

# Một số phiên bản plspm không hỗ trợ max_iter và tol trong hàm khởi tạo
# Hãy thử bỏ hai tham số này nếu gặp lỗi
plspm_model = Plspm(
    df_items,   # dữ liệu chỉ gồm các item
    cfg,        # cấu hình mô hình
    Scheme.PATH  # scheme PATH (gần giống SmartPLS)
)

print("PLS-PM đã chạy xong.")


PLS-PM đã chạy xong.


In [52]:
# Ô 6: Lấy các bảng kết quả chính tương đương SmartPLS

outer_model_df   = plspm_model.outer_model()       # outer loadings, communalities,...
inner_summary_df = plspm_model.inner_summary()     # R2, communality, redundancy,...
path_coeff_df    = plspm_model.path_coefficients() # hệ số đường dẫn
effects_df       = plspm_model.effects()           # direct / indirect / total effects

outer_model_df.head(), inner_summary_df, path_coeff_df, effects_df.head()


(       weight   loading  communality  redundancy
 CS1  0.395145  0.795024     0.632062    0.470539
 CS2  0.415110  0.817222     0.667852    0.497183
 CS3  0.428129  0.809600     0.655453    0.487952
 CS4  0.593589  0.864300     0.747015    0.504707
 CS5  0.571200  0.852522     0.726794    0.491045,
           type  r_squared  r_squared_adj  block_communality  mean_redundancy  \
 CS  Endogenous   0.675631       0.662827           0.736905         0.497876   
 HA   Exogenous   0.000000       0.000000           0.573467         0.000000   
 HL  Endogenous   0.744450       0.736099           0.651789         0.485225   
 KQ   Exogenous   0.000000       0.000000           0.612733         0.000000   
 NL   Exogenous   0.000000       0.000000           0.612004         0.000000   
 QL   Exogenous   0.000000       0.000000           0.605374         0.000000   
 QT   Exogenous   0.000000       0.000000           0.631573         0.000000   
 
          ave  
 CS  0.736905  
 HA  0.573467  
 

In [53]:
# Ô 7: Xuất toàn bộ kết quả Task 3 ra file Excel

output_path = "./output/Task3_PLS_Results.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    outer_model_df.to_excel(writer, sheet_name="Outer_Model")
    inner_summary_df.to_excel(writer, sheet_name="Inner_Summary")
    path_coeff_df.to_excel(writer, sheet_name="Path_Coefficients")
    effects_df.to_excel(writer, sheet_name="Effects")

print("Đã lưu kết quả Task 3 vào:", output_path)


Đã lưu kết quả Task 3 vào: ./output/Task3_PLS_Results.xlsx


# Task 4

In [61]:
# Cell 1 — Dùng plspm (CHUẨN)
import pandas as pd
import numpy as np
from plspm.plspm import Plspm

In [65]:
# ==== Task 4: Chạy PLS-SEM đúng chuẩn với plspm ====
# 1. Đảm bảo đã import đúng các thư viện và cấu hình
import pandas as pd
import plspm.config as c
from plspm.plspm import Plspm
from plspm.scheme import Scheme
from plspm.mode import Mode
from plspm.scale import Scale

# 2. Đọc lại dữ liệu và mapping thang đo (nếu cần)
# (Giả sử đã có df và SCALES_TASK3 từ các cell trước)

# 3. Định nghĩa inner model (ma trận đường dẫn giữa các latent variables)
structure = c.Structure()
structure.add_path(["NL", "KQ", "QT", "QL", "HA"], ["HL"])
structure.add_path(["NL", "KQ", "QT", "QL", "HA", "HL"], ["CS"])
path_matrix = structure.path()

# 4. Tạo Config cho plspm (outer model)
cfg = c.Config(path_matrix, default_scale=Scale.NUM)
for lv_name, indicators in SCALES_TASK3.items():
    cfg.add_lv(lv_name, Mode.A, *[c.MV(col) for col in indicators])

# 5. Chuẩn bị dữ liệu chỉ gồm các biến quan sát (item)
all_items = [item for items in SCALES_TASK3.values() for item in items]
df_items = df[all_items].copy()

# 6. Chạy mô hình PLS-SEM
plspm_model = Plspm(
    df_items,
    cfg,
    Scheme.PATH
 )

print("PLS-SEM đã chạy xong.")

PLS-SEM đã chạy xong.


IndexError: At least one sheet must be visible